In [6]:
import tarfile
import zipfile
import shutil
import random
import os
import cv2
import pathlib
import numpy as np
import pandas as pd

import tensorflow as tf
import matplotlib.pyplot as plt
from numpy.random import seed
from google.colab import drive
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import mixed_precision
from tensorflow.keras import regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet import ResNet101

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import top_k_accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [7]:
random_seed = 42

seed(random_seed)
tf.random.set_seed(random_seed)
random.seed(random_seed)

#policy = mixed_precision.Policy('mixed_float16')
#mixed_precision.set_global_policy(policy)
#print('Compute dtype: %s' % policy.compute_dtype)
#print('Variable dtype: %s' % policy.variable_dtype)

In [8]:
def plot_history(history):
  fig = make_subplots(rows=1, cols=3, subplot_titles=["Loss", "Top-1 Accuracy", "Top-3 Accuracy"])

  fig.add_trace(
      go.Scatter(y=history.history['val_loss'], name="val_loss"),
      row = 1, col = 1
  )

  fig.add_trace(
      go.Scatter(y=history.history['loss'], name="train_loss"),
      row = 1, col = 1
  )

  fig.add_trace(
      go.Scatter(y=history.history['val_accuracy'], name="val_top-1_accuracy"),
      row = 1, col = 2
  )

  fig.add_trace(
      go.Scatter(y=history.history['accuracy'], name="train_top-1_accuracy"),
      row = 1, col = 2
  )

  fig.add_trace(
      go.Scatter(y=history.history['val_top_k_categorical_accuracy'], name="val_top-3_accuracy"),
      row = 1, col = 3
  )

  fig.add_trace(
      go.Scatter(y=history.history['top_k_categorical_accuracy'], name="train_top-3_accuracy"),
      row = 1, col = 3
  )

  fig.update_xaxes(title_text="Epoch")
  fig.update_yaxes(title_text="Loss", row=1, col=1)
  fig.update_yaxes(title_text="Top-1 Accuracy", row=1, col=2)
  fig.update_yaxes(title_text="Top-3 Accuracy", row=1, col=3)
  fig.show()

# Importazione dataset

In [9]:
%%capture
drive.mount('/content/gdrive', force_remount=True)
!tar --extract --verbose --file='/content/gdrive/MyDrive/Progetto_VIPM/dataset/train_balanced.tar'
!tar --extract --verbose --file='/content/gdrive/MyDrive/Progetto_VIPM/dataset/val.tar'
with zipfile.ZipFile("/content/gdrive/MyDrive/Progetto_VIPM/dataset/val_degraded.zip","r") as zip_ref:
    zip_ref.extractall()

In [10]:
csv_train_file = pd.read_csv("/content/gdrive/MyDrive/Progetto_VIPM/annot/train_balanced_info.csv", dtype=str)
csv_train_file.columns = ['filename', 'label']
parent_dir = "train_set/"
labels = csv_train_file['label']

# divido le immagini in sottocartelle in base alla classe
for label in labels:
    path = os.path.join(parent_dir, label)
    os.makedirs(path, exist_ok=True)    
for _, row in csv_train_file.iterrows():
    label = row['label']
    path = os.path.join(parent_dir, row['filename'])
    img_name = os.path.split(path)[-1]
    new_path = os.path.join(parent_dir, label, img_name)
    print(new_path)
    shutil.copy(path, new_path)

Output streaming troncato alle ultime 5000 righe.
train_set/170/aug-170-_82_4257681.jpg
train_set/170/aug-170-_83_8987260.jpg
train_set/170/aug-170-_84_133930.jpg
train_set/170/aug-170-_85_2024164.jpg
train_set/170/aug-170-_86_5453615.jpg
train_set/170/aug-170-_87_3420783.jpg
train_set/170/aug-170-_88_6841804.jpg
train_set/170/aug-170-_89_1478944.jpg
train_set/170/aug-170-_8_6383002.jpg
train_set/170/aug-170-_90_6268877.jpg
train_set/170/aug-170-_91_1256851.jpg
train_set/170/aug-170-_92_1427779.jpg
train_set/170/aug-170-_93_3004325.jpg
train_set/170/aug-170-_94_5153235.jpg
train_set/170/aug-170-_95_935538.jpg
train_set/170/aug-170-_96_9366575.jpg
train_set/170/aug-170-_97_1262464.jpg
train_set/170/aug-170-_98_6717965.jpg
train_set/170/aug-170-_99_6880908.jpg
train_set/170/aug-170-_9_8561314.jpg
train_set/107/aug-107-_0_6404953.jpg
train_set/107/aug-107-_1_652592.jpg
train_set/107/aug-107-_2_5052261.jpg
train_set/107/aug-107-_3_2729665.jpg
train_set/107/aug-107-_4_7516430.jpg
train_set/

In [11]:
csv_test_clean_file = pd.read_csv("/content/gdrive/MyDrive/Progetto_VIPM/annot/val_info.csv", dtype=str)
csv_test_clean_file.columns = ['filename', 'label']
parent_dir = "val_set/"
labels = csv_test_clean_file['label']

# divido le immagini in sottocartelle in base alla classe
for label in labels:
    path = os.path.join(parent_dir, label)
    os.makedirs(path, exist_ok=True)    
for _, row in csv_test_clean_file.iterrows():
    label = row['label']
    path = os.path.join(parent_dir, row['filename'])
    img_name = os.path.split(path)[-1]
    new_path = os.path.join(parent_dir, label, img_name)
    print(new_path)
    shutil.copy(path, new_path)

Output streaming troncato alle ultime 5000 righe.
val_set/16/val_005256.jpg
val_set/119/val_005257.jpg
val_set/68/val_005258.jpg
val_set/130/val_005259.jpg
val_set/43/val_005260.jpg
val_set/197/val_005261.jpg
val_set/101/val_005262.jpg
val_set/55/val_005263.jpg
val_set/62/val_005264.jpg
val_set/49/val_005265.jpg
val_set/17/val_005266.jpg
val_set/204/val_005267.jpg
val_set/181/val_005268.jpg
val_set/122/val_005269.jpg
val_set/170/val_005270.jpg
val_set/66/val_005271.jpg
val_set/48/val_005272.jpg
val_set/96/val_005273.jpg
val_set/86/val_005274.jpg
val_set/144/val_005275.jpg
val_set/99/val_005276.jpg
val_set/4/val_005277.jpg
val_set/30/val_005278.jpg
val_set/177/val_005279.jpg
val_set/182/val_005280.jpg
val_set/201/val_005281.jpg
val_set/168/val_005282.jpg
val_set/187/val_005283.jpg
val_set/130/val_005284.jpg
val_set/30/val_005285.jpg
val_set/145/val_005286.jpg
val_set/153/val_005287.jpg
val_set/180/val_005288.jpg
val_set/160/val_005289.jpg
val_set/37/val_005290.jpg
val_set/52/val_005291.

In [12]:
csv_test_deg_file = pd.read_csv("/content/gdrive/MyDrive/Progetto_VIPM/annot/val_info.csv", dtype=str)
csv_test_deg_file.columns = ['filename', 'label']
parent_dir = "val_set_degraded/"
labels = csv_test_deg_file['label']

# divido le immagini in sottocartelle in base alla classe
for label in labels:
    path = os.path.join(parent_dir, label)
    os.makedirs(path, exist_ok=True)    
for _, row in csv_test_deg_file.iterrows():
    label = row['label']
    path = os.path.join(parent_dir, row['filename'])
    img_name = os.path.split(path)[-1]
    new_path = os.path.join(parent_dir, label, img_name)
    print(new_path)
    shutil.copy(path, new_path)

Output streaming troncato alle ultime 5000 righe.
val_set_degraded/16/val_005256.jpg
val_set_degraded/119/val_005257.jpg
val_set_degraded/68/val_005258.jpg
val_set_degraded/130/val_005259.jpg
val_set_degraded/43/val_005260.jpg
val_set_degraded/197/val_005261.jpg
val_set_degraded/101/val_005262.jpg
val_set_degraded/55/val_005263.jpg
val_set_degraded/62/val_005264.jpg
val_set_degraded/49/val_005265.jpg
val_set_degraded/17/val_005266.jpg
val_set_degraded/204/val_005267.jpg
val_set_degraded/181/val_005268.jpg
val_set_degraded/122/val_005269.jpg
val_set_degraded/170/val_005270.jpg
val_set_degraded/66/val_005271.jpg
val_set_degraded/48/val_005272.jpg
val_set_degraded/96/val_005273.jpg
val_set_degraded/86/val_005274.jpg
val_set_degraded/144/val_005275.jpg
val_set_degraded/99/val_005276.jpg
val_set_degraded/4/val_005277.jpg
val_set_degraded/30/val_005278.jpg
val_set_degraded/177/val_005279.jpg
val_set_degraded/182/val_005280.jpg
val_set_degraded/201/val_005281.jpg
val_set_degraded/168/val_0052

In [13]:
train_data_dir = pathlib.Path('train_set/')
seed_train_validation = 42 # deve essere lo stesso per train_ds e val_ds
shuffle_value = True
validation_split = 0.2
batch_size = 128

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_dir,
    labels="inferred",
    label_mode="categorical",
    validation_split=validation_split,
    subset="training",
    seed=seed_train_validation,
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_dir,
    labels="inferred",
    label_mode="categorical",
    validation_split=validation_split,
    subset="validation",
    seed=seed_train_validation,
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value)

Found 125499 files belonging to 251 classes.
Using 100400 files for training.
Found 125499 files belonging to 251 classes.
Using 25099 files for validation.


In [14]:
test_clean_data_dir = pathlib.Path('val_set/')
shuffle_value = True

test_clean_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_clean_data_dir,
    labels="inferred",
    label_mode="categorical",
    subset=None,
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value)

Found 11993 files belonging to 251 classes.


In [15]:
test_deg_data_dir = pathlib.Path('val_set_degraded/')
shuffle_value = True

test_deg_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_deg_data_dir,
    labels="inferred",
    label_mode="categorical",
    subset=None,
    image_size=(224,224),
    batch_size=batch_size,
    shuffle=shuffle_value)

Found 11993 files belonging to 251 classes.


In [16]:
def preprocess(images, labels):
  return tf.keras.applications.resnet.preprocess_input(images), labels

train_ds = train_ds.map(preprocess)
val_ds = val_ds.map(preprocess)
test_clean_ds = test_clean_ds.map(preprocess)
test_deg_ds = test_deg_ds.map(preprocess)

# Creazione modello

In [17]:
base_resnet101_model = ResNet101(weights="imagenet", include_top=True, input_shape=(224,224,3))
base_resnet101_model.summary()

179648224/179648224 [==============================] - 2s 0us/step
Model: "resnet101"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                       

In [18]:
trainable_layer = False
for layer in base_resnet101_model.layers:
  if layer.name == "conv4_block19_out":
    trainable_layer=True
    layer.trainable=True
  layer.trainable = trainable_layer

resnet101_model = Model(inputs=base_resnet101_model.input, 
                        outputs=base_resnet101_model.get_layer('avg_pool').output)
last_layer = base_resnet101_model.get_layer('avg_pool').output
flatten_layer = Flatten()(last_layer)
classification_layer = Dense(251, activation='softmax')(flatten_layer)
resnet101_model = Model(inputs=resnet101_model.input, 
                        outputs=classification_layer)
resnet101_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [19]:
def decay_schedule(epoch, lr):
    if (epoch % 10 == 0) and (epoch != 0):
        lr = lr * 0.1
    return lr
lr_scheduler = LearningRateScheduler(decay_schedule)

early_stop = EarlyStopping(monitor='val_loss', 
                           patience=25, 
                           verbose=1, 
                           restore_best_weights=True) 

checkpoint_filepath = '/content/gdrive/MyDrive/Progetto_VIPM/best_model_resnet101_balanced.h5'
best_model = ModelCheckpoint(checkpoint_filepath, 
                             verbose=1,
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             save_weights_only=False)

In [20]:
resnet101_model.compile(loss='categorical_crossentropy', 
                        optimizer=Adam(5e-5), 
                        metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3)])
history = resnet101_model.fit(train_ds, 
                              epochs=25, 
                              verbose=1, 
                              validation_data=val_ds,
                              callbacks=[lr_scheduler, early_stop, best_model])

Epoch 1/25
785/785 [==============================] - ETA: 0s - loss: 3.2537 - accuracy: 0.3254 - top_k_categorical_accuracy: 0.4825
Epoch 1: val_loss improved from inf to 2.38381, saving model to /content/gdrive/MyDrive/Progetto_VIPM/best_model_resnet101_balanced.h5
785/785 [==============================] - 823s 1s/step - loss: 3.2537 - accuracy: 0.3254 - top_k_categorical_accuracy: 0.4825 - val_loss: 2.3838 - val_accuracy: 0.4622 - val_top_k_categorical_accuracy: 0.6497 - lr: 5.0000e-05
Epoch 2/25
785/785 [==============================] - ETA: 0s - loss: 1.8694 - accuracy: 0.5640 - top_k_categorical_accuracy: 0.7369
Epoch 2: val_loss improved from 2.38381 to 2.13181, saving model to /content/gdrive/MyDrive/Progetto_VIPM/best_model_resnet101_balanced.h5
785/785 [==============================] - 808s 1s/step - loss: 1.8694 - accuracy: 0.5640 - top_k_categorical_accuracy: 0.7369 - val_loss: 2.1318 - val_accuracy: 0.5053 - val_top_k_categorical_accuracy: 0.6923 - lr: 5.0000e-05
Epoch 

In [21]:
plot_history(history)

In [22]:
score_clean = resnet101_model.evaluate(test_clean_ds, batch_size=batch_size)
print('test loss, test acc, test top-3 acc:', score_clean)

94/94 [==============================] - 62s 633ms/step - loss: 2.1909 - accuracy: 0.5856 - top_k_categorical_accuracy: 0.7684
test loss, test acc, test top-3 acc: [2.190885066986084, 0.5855916142463684, 0.7684482336044312]


In [23]:
score_deg = resnet101_model.evaluate(test_deg_ds, batch_size=batch_size)
print('test loss, test acc, test top-3 acc:', score_deg)

94/94 [==============================] - 61s 620ms/step - loss: 5.5716 - accuracy: 0.2120 - top_k_categorical_accuracy: 0.3402
test loss, test acc, test top-3 acc: [5.57164192199707, 0.21204034984111786, 0.34019845724105835]


In [ ]:
checkpoint_filepath = '/content/gdrive/MyDrive/Progetto_VIPM/best_model_resnet101_balanced.h5'
loaded_model = keras.models.load_model(checkpoint_filepath)